In [1]:
import warnings

import numpy as np

from seapodym_lmtl_python.configuration.no_transport.parameter_forcing import ForcingUnit
from seapodym_lmtl_python.configuration.no_transport.parameter_functional_group import (
    FunctionalGroupUnit,
    FunctionalGroupUnitMigratoryParameters,
    FunctionalGroupUnitRelationParameters,
)
from seapodym_lmtl_python.configuration.no_transport.parameters import (
    ForcingParameters,
    FunctionalGroups,
    NoTransportParameters,
)
from seapodym_lmtl_python.logging.custom_logger import set_critical, set_debug, set_error, set_verbose, set_warning
from seapodym_lmtl_python.model.no_transport_model import NoTransportModel
from seapodym_lmtl_python.configuration.no_transport.parameter_environment import (
    ChunkParameter,
    ClientParameter,
    EnvironmentParameter,
    OutputParameter,
    PreProductionParameter,
)

In [ ]:
path_forcing = (
    "/Users/ash/Documents/Workspaces/Data/phd/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1709010880580.nc"
)
path_bio = "/Users/ash/Documents/Workspaces/Data/phd/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1709007157635.nc"

> Set the verbosity of the program.


In [ ]:
MODE = 4

# ------------ #
if MODE == 1:
    set_debug()
elif MODE == 2:
    set_verbose()
elif MODE == 3:
    set_warning()
elif MODE == 4:
    set_error()
    warnings.filterwarnings("ignore")
else:
    set_critical()
    warnings.filterwarnings("ignore")

## Initialize the functional groups


In [ ]:
f_groups = []

# By functional group
day_layer = [1, 2, 2, 3, 3, 3]
night_layer = [1, 2, 1, 3, 2, 1]
energy_transfert = [0.17, 0.1, 0.22, 0.18, 0.13, 0.2]
# tr_max = [20, 30, 40, 50, 60, 70] # larger dataset
tr_max = [2, 3, 4, 5, 6, 7]

for dl, nl, ec, trm in zip(day_layer, night_layer, energy_transfert, tr_max):
    fg_f_param = FunctionalGroupUnitRelationParameters(
        inv_lambda_max=2109,
        inv_lambda_rate=0.125,
        temperature_recruitment_rate=-0.125,
        cohorts_timesteps=np.arange(1, trm + 1),
        temperature_recruitment_max=np.sum(np.arange(1, trm + 1)),
    )
    fg_m_param = FunctionalGroupUnitMigratoryParameters(
        day_layer=dl,
        night_layer=nl,
    )
    f_groups.append(
        FunctionalGroupUnit(
            name=f"D{dl}N{nl}",
            migratory_type=fg_m_param,
            functional_type=fg_f_param,
            energy_transfert=0.0042 * ec,
        )
    )
f_groups = FunctionalGroups(functional_groups=f_groups)

p_param = ForcingParameters(
    temperature=ForcingUnit.from_path(forcing=path_forcing, name="T", resolution=0.08333),
    primary_production=ForcingUnit.from_path(forcing=path_bio, name="npp", resolution=0.08333),
)

parameters = NoTransportParameters(
    functional_groups_parameters=f_groups,
    forcing_parameters=p_param,
    environment_parameters=EnvironmentParameter(
        chunk=ChunkParameter(functional_group=1),
        client=ClientParameter(n_workers=4, threads_per_worker=1, memory_limit="4GB"),
        output=OutputParameter(pre_production=PreProductionParameter()),
    ),
)

## Initialize the model


In [ ]:
my_class = NoTransportModel(configuration=parameters)
my_class.generate_configuration()
my_class.state

In [ ]:
my_class.initialize()
my_class.client

## Compute pre-production

This part is computed before the production step. All part of the process that can be pre-computed in parallele should be done here.


In [ ]:
my_class.pre_production()

## Compute production

This step is time dependent.


In [ ]:
my_class.production()

## Here we compute the output of the model


In [ ]:
my_class.post_production()

## Plotting the results


In [ ]:
biomass = my_class.state.biomass.load()

In [ ]:
biomass.cf.sum(("X", "Y")).cf.plot.line(x="T", hue="functional_group")

In [ ]:
my_class.state